# Summary

Recently (possibly since upgrading to openai 0.18.1?), querying openai causes invalid URL errors. Here are some notes from yesterday's troubleshooting session.

- curl works, python doesn't.
- Not just codex, now nox openai engines work w/ python. Maybe due to updating pip package? Temporarily upped billing limit to try other models.
- Restarted kernel and gpt query works again w/ ada. BUT after I import openai explicitly, that fails too. That must be a clue.
- Tried uninstalling, reinstalling, opened new tmux pane. Still same error.
- Tried deleting 'openai' object and then importing jabberwocky. This does work!?
- If I re-import openai after that, gpt.query still works. But openai.completion while codex does not.
- If I import openai FROM jabberwocky openai_utils, codex query still fails. But gpt.query works. And openai.Completion works w/ engine ada!
- Conclusion: maybe it is codex-specific then?

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path

from jabberwocky.config import C
from htools import *

In [3]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [4]:
j_kwargs = {'prompt': 'a', 'max_tokens': 1, 'engine_i': 0}
ada_kwargs = {'prompt': 'a', 'max_tokens': 1, 'engine': 'text-ada-001'}
code_kwargs = {'prompt': 'a', 'max_tokens': 1, 'engine': 'code-davinci-001'}

## Jabberwocky

In [11]:
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    GPTBackend
import jabberwocky.openai_utils as oautils

In [6]:
gpt = GPTBackend()
gpt

GPTBackend <current_name: openai>

In [8]:
res = gpt.query(**j_kwargs)

{'max_tokens': 1, 'engine_i': 0, 'prompt': 'a', 'meta': {'backend_name': 'openai', 'query_func': 'query_gpt3', 'datetime': 'Sun Apr 24 15:33:06 2022'}}


In [9]:
res

(['.'],
 [{'text': '.',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length',
   'prompt_index': 0}])

## openai 0.6.2

In [4]:
import openai

In [13]:
openai.version.VERSION

In [13]:
with open('/Users/hmamin/.openai', 'r') as f:
    openai.api_key = f.read().strip()

In [14]:
# oautils.openai == openai

In [15]:
res = openai.Completion.create(**code_kwargs)

In [16]:
res

<OpenAIObject text_completion id=cmpl-50fC98UoO0PUPIuYHcSJVxsANj2ej at 0x11d047b48> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1650840037,
  "id": "cmpl-50fC98UoO0PUPIuYHcSJVxsANj2ej",
  "model": "code-davinci:001",
  "object": "text_completion"
}

## openai 0.18.1

In [5]:
import openai

In [12]:
openai.version.VERSION

'0.18.1'

In [6]:
with open('/Users/hmamin/.openai', 'r') as f:
    openai.api_key = f.read().strip()

In [7]:
# oautils.openai == openai

In [8]:
res = openai.Completion.create(**code_kwargs)

In [9]:
res

<OpenAIObject text_completion id=cmpl-50fDMA9lymu1lk7W4Cyo6GRwZ9QSW at 0x12106caf0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "ns"
    }
  ],
  "created": 1650840112,
  "id": "cmpl-50fDMA9lymu1lk7W4Cyo6GRwZ9QSW",
  "model": "code-davinci:001",
  "object": "text_completion"
}

In [14]:
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    GPTBackend
import jabberwocky.openai_utils as oautils

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.


In [15]:
gpt = GPTBackend()
gpt

GPTBackend <current_name: openai>

In [16]:
res = gpt.query(**j_kwargs)

{'max_tokens': 1, 'engine_i': 0, 'prompt': 'a', 'meta': {'backend_name': 'openai', 'query_func': 'query_gpt3', 'datetime': 'Sun Apr 24 15:42:59 2022'}}


In [17]:
res

([''],
 [{'text': '\n',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length',
   'prompt_index': 0}])

## Takeaways

Could not reproduce error with either old or new version of openai package. Maybe ipython handles things differently from jupyter?

Update: could not reproduce in ipython either. No new openai version has been released in the last few days so it's not like they fixed something. Maybe it was an autoreload thing?